# JSON Parsing and Processing

In [ ]:
import json
import os
os.makedirs("data/json",exist_ok=True)

## Create JSON

In [ ]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [ ]:
json_data

In [ ]:
with open('data/json/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [ ]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

## Json Processing Strategy

## Method 1 - JSONLoader

In [ ]:
from langchain_community.document_loaders import JSONLoader
import json

In [ ]:
# Extract employee information
employee_loader = JSONLoader(
    file_path='data/json/company_data.json',
    jq_schema='.employees[]',  # jq query to extract each employee
    text_content=False  # Get full JSON objects
)
employee_docs = employee_loader.load()
print(f"Loaded {len(employee_docs)} employee documents")
print(f"First employee: {employee_docs[0].page_content[:200]}...")
print(employee_docs)

In [ ]:
# Method 2: Custom JSON processing for complex structures
from typing import List
from langchain_core.documents import Document

print("\n2️⃣ Custom JSON Processing")

def process_json_intelligently(filepath: str) -> List[Document]:
    """Process JSON with intelligent flattening and context preservation"""
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    documents = []
    
    # Strategy 1: Create documents for each employee with full context
    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"
        
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)

    return documents

In [ ]:
process_json_intelligently("data/json/company_data.json")